In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.data.path.append('C:/Users/vikra/nltk_data') 

import pandas as pd
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

In [27]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Test
print(word_tokenize("This is a test sentence."))


['This', 'is', 'a', 'test', 'sentence', '.']


In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')

df

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19995,this app is fricken stupid.it froze on the kin...,0
19996,Please add me!!!!! I need neighbors! Ginger101...,1
19997,love it! this game. is awesome. wish it had m...,1
19998,I love love love this app on my side of fashio...,1


In [34]:
# create preprocess_text function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

df_sample = df.head(100)  # Try 100 rows first to shorten runtime
df_sample['reviewText'] = df_sample['reviewText'].apply(preprocess_text)
df_sample

C:\Users\vikra\AppData\Local\Temp\ipykernel_17152\3573559735.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['reviewText'] = df_sample['reviewText'].apply(preprocess_text)


,reviewText,Positive
0,one best apps acording bunch people agree bomb...,1
1,pretty good version game free . lot different ...,1
2,really cool game . bunch level find golden egg...,1
3,"silly game frustrating , lot fun definitely re...",1
4,terrific game pad . hr fun . grandkids love . ...,1
...,...,...
95,"'ll admit , 'm late game getting wonderful gam...",1
96,"love angry bird , issue ad supported game . re...",0
97,love game level never time enjoy far get anoth...,0
98,see nothing wrong game busy n't think 's fun ....,0


In [39]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment


df_sample['sentiment'] = df_sample['reviewText'].apply(get_sentiment)
df_sample

C:\Users\vikra\AppData\Local\Temp\ipykernel_17152\2430842233.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['sentiment'] = df_sample['reviewText'].apply(get_sentiment)


,reviewText,Positive,sentiment
0,one best apps acording bunch people agree bomb...,1,1
1,pretty good version game free . lot different ...,1,1
2,really cool game . bunch level find golden egg...,1,1
3,"silly game frustrating , lot fun definitely re...",1,1
4,terrific game pad . hr fun . grandkids love . ...,1,1
...,...,...,...
95,"'ll admit , 'm late game getting wonderful gam...",1,1
96,"love angry bird , issue ad supported game . re...",0,1
97,love game level never time enjoy far get anoth...,0,1
98,see nothing wrong game busy n't think 's fun ....,0,1


In [41]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df_sample['Positive'], df_sample['sentiment']))

[[ 4 14]
 [ 2 80]]


In [42]:
from sklearn.metrics import classification_report
print(classification_report(df_sample['Positive'], df_sample['sentiment']))

              precision    recall  f1-score   support

           0       0.67      0.22      0.33        18
           1       0.85      0.98      0.91        82

    accuracy                           0.84       100
   macro avg       0.76      0.60      0.62       100
weighted avg       0.82      0.84      0.81       100

